# Fine-Tuning `DeepSeek-R1-Distill-Qwen` with KTO

## **Introduction**
In the rapidly evolving field of artificial intelligence, aligning large language models **(DeepSeek-R1-Distill-Qwen-1.5B)** with human preferences has become a critical challenge. Traditional methods, such as cross-entropy minimization, often fail to capture the nuanced ways in which humans perceive and evaluate information. Drawing inspiration from Kahneman and Tversky's prospect theory, which highlights the biased yet predictable nature of human decision-making, a novel approach called **Kahneman-Tversky Optimization (KTO)** has emerged. KTO leverages the principles of prospect theory to create human-aware loss functions (HALOs) that align LLMs more effectively with human feedback. Unlike preference-based methods like Direct Preference Optimization (DPO), KTO does not require costly preference data; instead, it relies on a simple binary signal indicating whether an output is desirable or undesirable. This makes KTO not only more efficient but also more practical for real-world applications where preference data is scarce. In this example, we demonstrate how to implement KTO for fine-tuning a language model using the `unsloth` library, showcasing its potential to improve model alignment with human values.

## **Overview of `DeepSeek-R1-Distill-Qwen`**
- **Base Model**: DeepSeek-R1-Distill-Qwen utilizes the Qwen2.5-32B model as its foundation. This model has been fine-tuned using samples generated by the more powerful DeepSeek-R1, which itself integrates advanced reasoning capabilities through a combination of reinforcement learning (RL) and cold-start data[1][2].
- **Performance**: The distilled version has demonstrated superior performance on various reasoning benchmarks, outperforming other models like OpenAI's o1-mini. Notably, it achieved state-of-the-art results across multiple tasks, showcasing its effectiveness in reasoning and problem-solving[2][6].
- **Training Methodology**: The distillation process allows for the transfer of reasoning patterns from larger models to smaller, more efficient ones. This approach not only enhances the performance of smaller models but also makes them more accessible for practical applications[1][3].

### Key Features
- **Reasoning Capability**: DeepSeek-R1-Distill-Qwen excels in tasks that require complex reasoning, such as mathematics and coding challenges. It has been shown to outperform traditional models significantly in these areas[1][4].
- **Open Source Availability**: The model is open-sourced, providing researchers and developers with access to its architecture and capabilities. This encourages experimentation and further development within the AI community[2][5].
- **Applications**: Users can deploy this model similarly to other popular LLMs like Qwen or Llama, making it versatile for various applications, including chatbots, coding assistants, and educational tools[3][4].


## **Step 1: Install Required Packages**
The code installs and upgrades necessary libraries for efficient model training and inference. This includes `unsloth` for fast training, `torch` for GPU operations, and `flash-attn` for optimized attention computation on compatible GPUs.

In [ ]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Install Flash Attention 2 for GPUs with CUDA capability >= 8 (e.g., A100, H100)
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"  # Flash Attention 2 for faster training

## **Step 2: Import Necessary Libraries**
The code imports libraries required for model training, dataset handling, and inference. Key libraries include `unsloth` for efficient model loading, `datasets` for dataset management, and `trl` for KTO training.

In [ ]:
import torch  # For GPU operations and tensor computations
import os     # For file and directory operations
import re     # For regular expressions
from typing import List, Literal, Optional  # For type hints
from datasets import load_dataset
from unsloth import FastLanguageModel, is_bfloat16_supported  # For efficient model loading and training
from trl import KTOConfig, KTOTrainer                         # For KTO training

## **Step 3: Model Loading and Configuration**
The code sets up the model and tokenizer, configures quantization, and applies a default chat template if missing. It uses 4-bit quantization to reduce memory usage and auto-detects the appropriate data type (float16 or bfloat16).

In [ ]:
# Set basic parameters
max_seq_length = 4096  # Maximum sequence length the model can handle
dtype = None           # Auto-detect data type (float16 for Tesla T4/V100, bfloat16 for Ampere+ GPUs)
load_in_4bit = True    # Use 4-bit quantization to reduce memory usage

# Load the pre-trained model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",
    max_seq_length=max_seq_length,  # Set max sequence length
    dtype=dtype,                    # Auto-detect data type
    load_in_4bit=load_in_4bit,      # Enable 4-bit quantization
    # token="hf_...",                # Use this if accessing gated models (e.g., LLaMA 2)
)

# Add a default chat template if missing
if tokenizer.chat_template is None:
    DEFAULT_CHAT_TEMPLATE = """
    {% for message in messages %}
    {% if message['role'] == 'user' %}
    {{ '<|user|>\n' + message['content'] + eos_token }}
    {% elif message['role'] == 'system' %}
    {{ '<|system|>\n' + message['content'] + eos_token }}
    {% elif message['role'] == 'assistant' %}
    {{ '<|assistant|>\n' + message['content'] + eos_token }}
    {% endif %}
    {% if loop.last and add_generation_prompt %}
    {{ '<|assistant|>' }}
    {% endif %}
    {% endfor %}
    """
    tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE  # Apply the default template

## **Step 4: Dataset Preparation and Processing**
The code defines a function to apply a chat template to dataset examples and loads the KTO dataset. It also selects a subset of the dataset for faster training.

In [ ]:
# Function to apply chat template to dataset examples
def apply_chat_template(
    example, tokenizer, task: Literal["sft", "generation", "rm", "kto"] = "sft", assistant_prefix="<|assistant|>\n"
):
    def _strip_prefix(s, pattern):
        # Remove a prefix from a string using regex
        return re.sub(f"^{re.escape(pattern)}", "", s)

    if task in ["sft", "generation"]:
        messages = example["messages"]
        # Add an empty system message if none exists
        if messages[0]["role"] != "system":
            messages.insert(0, {"role": "system", "content": ""})
        example["text"] = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True if task == "generation" else False
        )
    elif task == "rm":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            chosen_messages = example["chosen"]
            rejected_messages = example["rejected"]
            # Add an empty system message if none exists
            if chosen_messages[0]["role"] != "system":
                chosen_messages.insert(0, {"role": "system", "content": ""})
            if rejected_messages[0]["role"] != "system":
                rejected_messages.insert(0, {"role": "system", "content": ""})
            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
        else:
            raise ValueError(
                f"Could not format example as dialogue for `rm` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )
    elif task == "dpo":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            prompt_messages = [[msg for msg in example["chosen"] if msg["role"] == "user"][0]]
            # Insert system message
            if example["chosen"][0]["role"] != "system":
                prompt_messages.insert(0, {"role": "system", "content": ""})
            else:
                prompt_messages.insert(0, example["chosen"][0])
            chosen_messages = example["chosen"][1:]
            rejected_messages = example["rejected"][1:]
            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
            example["text_prompt"] = tokenizer.apply_chat_template(
                prompt_messages, tokenize=False, add_generation_prompt=True
            )
            example["text_chosen"] = _strip_prefix(example["text_chosen"], assistant_prefix)
            example["text_rejected"] = _strip_prefix(example["text_rejected"], assistant_prefix)
        else:
            raise ValueError(
                f"Could not format example as dialogue for `dpo` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )
    elif task == "kto":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            prompt_messages = [[msg for msg in example["chosen"] if msg["role"] == "user"][0]]
            chosen_messages = prompt_messages + [msg for msg in example["chosen"] if msg["role"] == "assistant"]
            rejected_messages = prompt_messages + [msg for msg in example["rejected"] if msg["role"] == "assistant"]
            if "system" in example:
                chosen_messages.insert(0, {"role": "system", "content": example["system"]})
                rejected_messages.insert(0, {"role": "system", "content": example["system"]})
            example["text_chosen"] = _strip_prefix(tokenizer.apply_chat_template(chosen_messages, tokenize=False), assistant_prefix)
            example["text_rejected"] = _strip_prefix(tokenizer.apply_chat_template(rejected_messages, tokenize=False), assistant_prefix)
        else:
            raise ValueError(f"Could not format example as dialogue for `kto` task!")
    else:
        raise ValueError(
            f"Task {task} not supported, please ensure that the provided task is one of {['sft', 'generation', 'rm', 'dpo', 'kto']}"
        )
    return example

# Load the KTO dataset
raw_datasets = load_dataset("trl-lib/kto-mix-14k")  # Load the dataset
train_dataset = raw_datasets["train"]               # Use the training split

# Take a subset of the training data (1000 examples for faster training)
train_subset = train_dataset.select(range(1000))

## **Step 5: Model Training Setup**
The code configures LoRA for parameter-efficient fine-tuning and sets up the KTO trainer with training arguments. It also prints GPU memory stats to monitor resource usage.

In [ ]:
# Configure LoRA (Low-Rank Adaptation) for parameter-efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # Target layers
    lora_alpha=16,      # LoRA scaling factor
    lora_dropout=0,     # Dropout for LoRA layers
    bias="none",        # No bias for LoRA
    use_gradient_checkpointing="unsloth",  # Enable gradient checkpointing for memory efficiency
    random_state=3407,  # Random seed for reproducibility
)

# Set up the KTO trainer with training arguments
kto_trainer = KTOTrainer(
    model=model,
    args=KTOConfig(
        per_device_train_batch_size=4,    # Batch size per GPU
        gradient_accumulation_steps=2,    # Gradient accumulation for larger effective batch size
        num_train_epochs=1,               # Number of training epochs
        learning_rate=5e-7,               # Learning rate
        fp16=not is_bfloat16_supported(), # Use FP16 if BF16 is not supported
        bf16=is_bfloat16_supported(),     # Use BF16 if supported
        output_dir="outputs",             # Directory to save outputs
        logging_steps=1,                  # Log training progress every step
        optim="adamw_8bit",               # Use 8-bit AdamW optimizer
        weight_decay=0.01,                # Weight decay for regularization
        lr_scheduler_type="cosine",       # Cosine learning rate scheduler
        warmup_ratio=0.1,                 # Warmup ratio for learning rate
        seed=42,                          # Random seed
        report_to="none",                 # Disable external logging (e.g., WandB)
    ),
    train_dataset=train_subset,           # Training dataset
    processing_class=tokenizer,           # Tokenizer for processing data
)

# Print GPU memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Train the model
kto_trainer.train()

## **Step 6: Model Saving and Export**
The code saves the fine-tuned model and tokenizer locally and provides options to save in merged formats (16-bit or 4-bit) or push to Hugging Face Hub. It also supports converting the model to GGUF format for use with `llama.cpp`.

In [ ]:
# Save the fine-tuned model and tokenizer locally
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

# Save the merged model in 16-bit or 4-bit format (optional)
if False:  # Set to True to enable
    model.save_pretrained_merged("merged_model", tokenizer, save_method="merged_16bit")  # 16-bit merged model
    # model.save_pretrained_merged("merged_model", tokenizer, save_method="merged_4bit")   # 4-bit merged model

# Push the model to Hugging Face Hub (optional)
if False:  # Set to True to enable
    model.push_to_hub_merged("your_name/model", tokenizer, save_method="merged_16bit", token="...")  # Upload to Hugging Face Hub

# Convert the model to GGUF format for llama.cpp (optional)
if False:
    !git clone https://github.com/ggerganov/llama.cpp  # Clone llama.cpp
    !cd llama.cpp && make                              # Build llama.cpp
    !python3 llama.cpp/convert.py merged_model/ --outfile model-unsloth.gguf   # Convert to GGUF format
    !./llama.cpp/quantize model-unsloth.gguf model-unsloth-Q4_K_M.gguf Q4_K_M  # Quantize the model

## **Step 7: Inference and Response Generation**
The code defines a function to generate responses from the fine-tuned model and tests it on a set of sample questions.

In [ ]:
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    mapping = {"role": "role", "content": "content", "user": "user", "assistant": "assistant"},
)

FastLanguageModel.for_inference(model)

def generate_response(message):
    print("\n" + "="*50 + "\nQUESTION:\n" + "="*50)
    print(message + "\n")
    print("-"*50 + "\nRESPONSE:\n" + "-"*50)

    messages = [{"content": message, "role": "user"}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt"
    ).to("cuda")

    text_streamer = TextStreamer(tokenizer, skip_special_tokens=True, skip_prompt=True)
    outputs = model.generate(
        input_ids=inputs,
        streamer=text_streamer,
        temperature=0.1,      # Sampling temperature
        max_new_tokens=1024,  # Maximum tokens to generate
        use_cache=True        # Use caching for faster generation
    )
    return outputs

# Test questions
questions = [
    "Q:Question: how old julio cesar chavez when he fought de la hoya I found the following answer on Google: He holds records for most successful consecutive defenses of world titles (27), most title fights (37), most title-fight victories (31) and he is after Joe Louis with (23) for most title defenses won by knockout (21). Is that a correct answer? Yes or no.\nA:",
    "Q:Information: - The Assistant Secretary of Defense for Health Affairs (ASD(HA)) is chartered under United States Department of Defense Directive (DoDD) 5136.1 in 1994. This DoDD states that the ASD(HA) is the principal advisor to the U.S. Secretary of Defense on all \"DoD health policies, programs and activities.\" In addition to exercising oversight of all DoD health resources, ASD(HA) serves as director of the Tricare Management Activity. - The Department of the Air Force (DAF) is one of the three Military Departments within the Department of Defense of the United States of America. The Department of the Air Force was formed on September 18, 1947, per the National Security Act of 1947 and it includes all elements and units of the United States Air Force (USAF). - The Surgeon General of the Air Force is the senior-most Medical Service officer in the United States Department of the Air Force. In recent times, this has been a Lieutenant General who serves as head of the United States Air Force Medical Service (AFMS). The Surgeon General is usually the senior Medical Corps officer, but acting surgeons general have been from other branches of the medical service. - Lieutenant general, lieutenant-general and similar (abbrev Lt Gen, LTG and similar) is a three-star military rank (NATO code OF-8) used in many countries. The rank traces its origins to the Middle Ages, where the title of lieutenant general was held by the second in command on the battlefield, who was normally subordinate to a captain general. - The United States Air Force (USAF) is the aerial warfare service branch of the United States Armed Forces and one of the seven American uniformed services. Initially part of the United States Army, the USAF was formed as a separate branch of the military on 18 September 1947 under the National Security Act of 1947. It is the most recent branch of the U.S. military to be formed, and is the largest and one of the world's most technologically advanced air forces. The USAF articulates its core functions as Nuclear Deterrence Operations, Special Operations, Air Superiority, Global Integrated ISR, Space Superiority, Command and Control, Cyberspace Superiority, Personnel Recovery, Global Precision Attack, Building Partnerships, Rapid Global Mobility and Agile Combat Support. - Lieutenant General James Gordon Roudebush , USAF , ( born February 24 , 1948 ) was the 19th Surgeon General of the United States Air Force , Headquarters U.S. Air Force , Washington , D.C. General Roudebush served as functional manager of the U.S. Air Force Medical Service . In this capacity , he advised the Secretary of the Air Force and Air Force Chief of Staff , as well as the Assistant Secretary of Defense for Health Affairs on matters pertaining to the medical aspects of the air expeditionary force and the health of Air Force people . General Roudebush had authority to commit resources worldwide for the Air Force Medical Service , to make decisions affecting the delivery of medical services , and to develop plans , programs and procedures to support worldwide medical service missions . He exercised direction , guidance and technical management of more than 42,400 people assigned to 74 medical facilities worldwide . A native of Gering , Nebraska , Roudebush entered the Air Force in 1975 after receiving a Bachelor of Medicine degree from the University of Nebraska at Lincoln , and a Doctor of Medicine degree from the University of Nebraska College of Medicine . He completed residency training in family practice at the Wright - Patterson Air Force Medical Center , Ohio , in 1978 , and aerospace medicine at Brooks Air Force Base , Texas , in 1984 . He commanded a wing clinic and wing hospital before becoming Deputy Commander of the Air Force Materiel Command Human Systems Center . He has served as Command Surgeon for U.S. Central Command , Pacific Air Forces , U.S. Transportation Command and Headquarters Air Mobility Command . Prior to his selection as the 19th Surgeon General , he served as the Deputy Surgeon General of the U.S. Air Force . He retired from the U.S. Air Force on October 1 , 2009 . After reading the paragraphs above, choose the best answer for the entity that related to 'james g. roudebush' with the relationship of 'occupation'. Choices: - advisor - army - captain - general - lieutenant - military - officer - secretary - surgeon - united states of america\nA:",
    "If But slowly and doggedly he went on sawing to and fro., can we conclude that \"It was difficult to keep sawing.\"?",
    "You are given a list of queries separated by new line. Your job is to answer with the query that is the most well-formed or well-structured query in terms of grammar, punctuations, or spelling errors.\nQ: How do you set the alarm on the prospirit watch ?\nThe allies tried to regain access to the battle of Gallipoli ?\nWhat is scooter smith real phone number not a fake one ?\nLaw of Supply and Demand defined ?\nA:",
    "How does the sentence end? See options at the end\n\nThe woman tried to put the books on the couches but the \n\nAvailable options: - couches were too large. - books were too large.",
]

# Generate responses for each question
for question in questions:
    generate_response(question)

---

## **Conclusion**
The implementation of **Kahneman-Tversky Optimization (KTO)** represents a significant step forward in aligning large language models with human preferences. By incorporating insights from prospect theory, KTO provides a more human-aware approach to model training, eliminating the need for expensive and scarce preference data. In this example, we fine-tuned a 1.5B parameter model using KTO, demonstrating its effectiveness in generating human-aligned responses. The use of 4-bit quantization, LoRA adapters, and Flash Attention 2 further highlights the efficiency and scalability of this approach. As AI systems continue to play an increasingly important role in society, methods like KTO will be essential for ensuring that these systems align with human values and expectations. Future work could explore the application of KTO to larger models and diverse datasets, as well as its integration with other alignment techniques to further enhance performance and usability.